In [1]:
import sys
sys.path.append("../src")

import pandas as pd

import plotly.graph_objs as go
import plotly.offline as py

import utilities as u
import constants as c

In [2]:
import io
import base64

In [7]:
df = pd.read_csv(c.os.FILE_DATA_SAMPLE, sep=";", index_col=0)

In [8]:
b64 = u.uos.df_to_b64(df)

In [23]:
u.uos.b64_to_df(b64.decode()).head()

,Note,Amount,Category,Date
ID,,,,
1,Apartment,-500,Rent,2017/01/01
2,Apartment,-500,Rent,2017/02/01
3,Apartment,-500,Rent,2017/03/01
4,Apartment,-500,Rent,2017/04/01
5,Apartment,-500,Rent,2017/05/01
